In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
cd /content/drive/MyDrive/DL_Project

/content/drive/MyDrive/DL_Project


In [3]:
!pip install tensorboardX
!pip install pyyaml==5.4.1
!pip install deeplake

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
%reload_ext autoreload
%autoreload 2

import os
import random
import time
import shutil
from argparse import ArgumentParser

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision.utils as vutils
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter

from trainer import Trainer
from data.dataset import Dataset, DeeplakeDataset
from utils.tools import get_config, random_bbox, mask_image, is_image_file, default_loader, normalize, get_model_list
from utils.logger import get_logger
from model.networks import Generator

import matplotlib.pyplot as plt
import deeplake

In [5]:
print(torch.cuda.get_device_name())
print(torch.cuda.is_available())
print(torch.cuda.device_count())

Tesla T4
True
1


### Train the Model

In [ ]:
def train():
    # args = parser.parse_args()
    # config = get_config(args.config)

    config_path = 'configs/config.yaml' # Edited by Manoj
    config = get_config(config_path)
    seed = None

    # CUDA configuration
    cuda = config['cuda']
    device_ids = config['gpu_ids']
    if cuda:
        os.environ['CUDA_VISIBLE_DEVICES'] = ','.join(str(i) for i in device_ids)
        device_ids = list(range(len(device_ids)))
        config['gpu_ids'] = device_ids
        cudnn.benchmark = True

    # Configure checkpoint path
    checkpoint_path = os.path.join('checkpoints',
                                   config['dataset_name'],
                                   config['mask_type'] + '_' + config['expname'])
    print("Checkpoint Path is:", checkpoint_path)
    if not os.path.exists(checkpoint_path):
        os.makedirs(checkpoint_path)
    # shutil.copy(args.config, os.path.join(checkpoint_path, os.path.basename(args.config)))
    shutil.copy(config_path, os.path.join(checkpoint_path, os.path.basename(config_path))) # Edited by Manoj
    writer = SummaryWriter(logdir=checkpoint_path)
    logger = get_logger(checkpoint_path)    # get logger and configure it at the first call

    # logger.info("Arguments: {}".format(args))
    # Set random seed
    if seed is None:
        seed = random.randint(1, 10000)
    logger.info("Random seed: {}".format(seed))
    random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

    # Log the configuration
    logger.info("Configuration: {}".format(config))

    try:  # for unexpected error logging
        # Load the dataset
        logger.info("Training on dataset: {}".format(config['dataset_name']))
        # train_dataset = Dataset(data_path=config['train_data_path'],
        #                         with_subfolder=config['data_with_subfolder'],
        #                         image_shape=config['image_shape'],
        #                         random_crop=config['random_crop'])
        
        train_dataset = DeeplakeDataset(data_path=config['train_data_path'],
                        with_subfolder=config['data_with_subfolder'],
                        image_shape=config['image_shape'],
                        random_crop=config['random_crop'])

        # Generate a random subset of indices
        random.seed(seed)
        num_images = len(train_dataset.data)

        subset_indices = random.sample(range(num_images), k=config['data_subset_size'])

        # val_dataset = Dataset(data_path=config['val_data_path'],
        #                       with_subfolder=config['data_with_subfolder'],
        #                       image_size=config['image_size'],
        #                       random_crop=config['random_crop'])


        # Load DataLoader with the above Random Sample
        train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                                   batch_size=config['batch_size'],
                                                   shuffle=False,
                                                   num_workers=config['num_workers'],
                                                   sampler = torch.utils.data.SubsetRandomSampler(subset_indices))
        print("Total No. of Batches =", len(train_loader))
        
        # val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
        #                                           batch_size=config['batch_size'],
        #                                           shuffle=False,
        #                                           num_workers=config['num_workers'])

        # Define the trainer
        trainer = Trainer(config)
        # logger.info("\n{}".format(trainer.netG))
        # print("NETG done")
        # logger.info("\n{}".format(trainer.localD))
        # print("LocalD done")
        # logger.info("\n{}".format(trainer.globalD))
        # print("GlobalD done")

        if cuda:
            trainer = nn.parallel.DataParallel(trainer, device_ids=device_ids)
            trainer_module = trainer.module
        else:
            trainer_module = trainer

        # Get the resume iteration to restart training
        start_iteration = trainer_module.resume(checkpoint_path, config['resume']) if config['resume'] else 1

        iterable_train_loader = iter(train_loader)

        time_count = time.time()

        for iteration in range(start_iteration, config['niter'] + 1):
        # for iteration in range(5700, config['niter'] + 1):
            try:
                ground_truth = next(iterable_train_loader)
            except StopIteration:
                iterable_train_loader = iter(train_loader)
                ground_truth = next(iterable_train_loader)
            
            ## Edited by Manoj
            # print("Original Image Example")
            # img = ground_truth[0]
            # plt.imshow(img.permute(1,2,0))
            # plt.show()

            # Prepare the inputs
            bboxes = random_bbox(config, batch_size=ground_truth.size(0))
            x, mask = mask_image(ground_truth, bboxes, config)

            ## Edited by Manoj
            # print("Masked Image Example")
            # img = x[0]
            # plt.imshow(img.permute(1,2,0))
            # plt.show()

            # print("Mask Creation")
            # img = mask[0]
            # plt.imshow(img.permute(1,2,0))
            # plt.show()

            if cuda:
                x = x.cuda()
                mask = mask.cuda()
                ground_truth = ground_truth.cuda()

            ###### Forward pass ######
            compute_g_loss = iteration % config['n_critic'] == 0
            losses, inpainted_result, offset_flow = trainer(x, bboxes, mask, ground_truth, compute_g_loss)
            # Scalars from different devices are gathered into vectors
            for k in losses.keys():
                if not losses[k].dim() == 0:
                    losses[k] = torch.mean(losses[k])

            ###### Backward pass ######
            # Update D
            trainer_module.optimizer_d.zero_grad()
            losses['d'] = losses['wgan_d'] + losses['wgan_gp'] * config['wgan_gp_lambda']
            losses['d'].backward()
            trainer_module.optimizer_d.step()

            # Update G
            if compute_g_loss:
                trainer_module.optimizer_g.zero_grad()
                losses['g'] = losses['l1'] * config['l1_loss_alpha'] \
                              + losses['ae'] * config['ae_loss_alpha'] \
                              + losses['wgan_g'] * config['gan_loss_alpha']
                losses['g'].backward()
                trainer_module.optimizer_g.step()

            # Log and visualization
            log_losses = ['l1', 'ae', 'wgan_g', 'wgan_d', 'wgan_gp', 'g', 'd']
            if iteration % config['print_iter'] == 0:
                time_count = time.time() - time_count
                speed = config['print_iter'] / time_count
                speed_msg = 'speed: %.2f batches/s ' % speed
                time_count = time.time()

                message = 'Iter: [%d/%d] ' % (iteration, config['niter'])
                for k in log_losses:
                    v = losses.get(k, 0.)
                    writer.add_scalar(k, v, iteration)
                    message += '%s: %.6f ' % (k, v)
                message += speed_msg
                logger.info(message)

            if iteration % (config['viz_iter']) == 0:
                viz_max_out = config['viz_max_out']
                if x.size(0) > viz_max_out:
                    viz_images = torch.stack([x[:viz_max_out], inpainted_result[:viz_max_out],
                                              offset_flow[:viz_max_out]], dim=1)
                else:
                    viz_images = torch.stack([x, inpainted_result, offset_flow], dim=1)
                viz_images = viz_images.view(-1, *list(x.size())[1:])
                vutils.save_image(viz_images,
                                  '%s/niter_%03d.png' % (checkpoint_path, iteration),
                                  nrow=3 * 4,
                                  normalize=True)

            # Save the model
            if iteration % config['snapshot_save_iter'] == 0:
                # trainer_module.save_model(checkpoint_path, iteration)
                trainer_module.save_model(checkpoint_path, 1000) # Keeping 1000 as constant to keep on replacing the same model file

    except Exception as e:  # for unexpected error logging
        logger.error("{}".format(e))
        raise e


In [ ]:
# train()

## Model Evaluation

In [23]:
def test_single(image = "/content/drive/MyDrive/DL_Project/examples/places/images/image0.jpg", 
                mask = None, 
                output = "/content/drive/MyDrive/DL_Project/examples/places/output/image0.jpg",
                masked_output_image = '/content/drive/MyDrive/DL_Project/examples/places/masks/image0.jpg'):
    
    """
    Function to evaluate the generative inpainting model on a single test image.
    image : path to the image
    mask : path to the input mask. If None, then random mask is generated.
    output : Path to store the output image
    masked_image_output: Path to store the masked image
    """
    
    # args = parser.parse_args()
    # config = get_config(args.config)

    config_path = 'configs/config.yaml'
    config = get_config(config_path)
    seed = 200

    # CUDA configuration
    cuda = config['cuda']
    device_ids = config['gpu_ids']
    if cuda:
        os.environ['CUDA_VISIBLE_DEVICES'] = ','.join(str(i) for i in device_ids)
        device_ids = list(range(len(device_ids)))
        config['gpu_ids'] = device_ids
        cudnn.benchmark = True

    # print("Arguments: {}".format(args))

    # Set random seed
    if seed is None:
        seed = random.randint(1, 10000)
    print("Random seed: {}".format(seed))
    random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

    print("Configuration: {}".format(config))

    try:  # for unexpected error logging
        with torch.no_grad():   # enter no grad context
            if is_image_file(image):
                if mask and is_image_file(mask):
                    # Test a single masked image with a given mask
                    x = default_loader(image)
                    mask = default_loader(mask)
                    x = transforms.Resize(config['image_shape'][:-1])(x)
                    x = transforms.CenterCrop(config['image_shape'][:-1])(x)
                    mask = transforms.Resize(config['image_shape'][:-1])(mask)
                    mask = transforms.CenterCrop(config['image_shape'][:-1])(mask)
                    x = transforms.ToTensor()(x)
                    mask = transforms.ToTensor()(mask)[0].unsqueeze(dim=0)
                    x = normalize(x)
                    x = x * (1. - mask)
                    x = x.unsqueeze(dim=0)
                    mask = mask.unsqueeze(dim=0)
                elif mask:
                    raise TypeError("{} is not an image file.".format(mask))
                else:
                    # Test a single ground-truth image with a random mask
                    ground_truth = default_loader(image)
                    ground_truth = transforms.Resize(config['image_shape'][:-1])(ground_truth)
                    ground_truth = transforms.CenterCrop(config['image_shape'][:-1])(ground_truth)
                    ground_truth = transforms.ToTensor()(ground_truth)
                    ground_truth = normalize(ground_truth)
                    ground_truth = ground_truth.unsqueeze(dim=0)
                    bboxes = random_bbox(config, batch_size=ground_truth.size(0))
                    x, mask = mask_image(ground_truth, bboxes, config)

                # Set checkpoint path
                # if not checkpoint_path:
                checkpoint_path = os.path.join('checkpoints',
                                                config['dataset_name'],
                                                config['mask_type'] + '_' + config['expname'])
                # else:
                #     checkpoint_path = checkpoint_path

                # Define the trainer
                netG = Generator(config['netG'], cuda, device_ids)
                # Resume weight
                # last_model_name = get_model_list(checkpoint_path, "gen", iteration=args.iter)
                last_model_name = get_model_list(checkpoint_path, "gen", iteration=1000) # Kept 1000 as static iteration name for model to keep on replacing the same model
                netG.load_state_dict(torch.load(last_model_name))
                model_iteration = int(last_model_name[-11:-3])
                print("Resume from {} at iteration {}".format(checkpoint_path, model_iteration))

                if cuda:
                    netG = nn.parallel.DataParallel(netG, device_ids=device_ids)
                    x = x.cuda()
                    mask = mask.cuda()

                # Inference
                x1, x2, offset_flow = netG(x, mask)
                inpainted_result = x2 * mask + x * (1. - mask)

                # Save final image
                vutils.save_image(inpainted_result, output, padding=0, normalize=True)
                print("Saved the inpainted result to {}".format(output))

                # Save mask
                vutils.save_image(x, masked_output_image, padding=0, normalize=True)
                print("Saved the masked result to {}".format(masked_output_image))

                # if args.flow:
                #     vutils.save_image(offset_flow, args.flow, padding=0, normalize=True)
                #     print("Saved offset flow to {}".format(args.flow))
            else:
                raise TypeError("{} is not an image file.".format)
        # exit no grad context
    except Exception as e:  # for unexpected error logging
        print("Error: {}".format(e))
        raise e

In [26]:
def test_all_images(input_dir, output_dir, mask_output_dir):
    """
    Generate inpainted output images for all images in the input directory and store in the output directory
    input_dir: path to the input directory
    output_dir: path to the output directory 
    masked_output_dir: path to the masked image
    """

    # create output directory if it does not exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # loop through each file in input directory
    for i, file_name in enumerate(os.listdir(input_dir)):
        # Generate image input and output path
        image_path = os.path.join(input_dir, file_name)
        output_image_path = os.path.join(output_dir, file_name)
        output_masked_path = os.path.join(mask_output_dir, file_name)
        print(i, file_name)
        test_single(image = image_path, mask = None, output = output_image_path, masked_output_image = output_masked_path)



In [ ]:
#### Run the evaluation

## Run all test images

# set input and output paths
input_dir = "/content/drive/MyDrive/DL_Project/examples/places/images"
output_dir = '/content/drive/MyDrive/DL_Project/examples/places/output'
mask_output_dir = '/content/drive/MyDrive/DL_Project/examples/places/masks'

test_all_images(input_dir, output_dir, mask_output_dir) 


## Run one single test image
# image_path = "/content/drive/MyDrive/DL_Project/data/places/places_1.jpg"
# output_path = "/content/drive/MyDrive/DL_Project/examples/places/output_1.png"

# test_single(image = image_path, mask = None, output = output_path)

## Rough

In [15]:
# import os
# import glob
# import numpy as np
# from PIL import Image

# # Set the input and output directories
# input_dir = "/content/drive/MyDrive/DL_Project/examples/places/images_npy"
# output_dir = '/content/drive/MyDrive/DL_Project/examples/places/images'

# # Find all .npy files in the input directory
# npy_files = glob.glob(os.path.join(input_dir, '*.npy'))

# # Loop through all .npy files and convert them to .jpg
# for i, npy_file in enumerate(npy_files):
#     # Load the .npy file as a NumPy array
#     try:
#       # print(i, "Image:", npy_file)
#       npy_array = np.load(npy_file)
#       # Convert the NumPy array to a PIL image
#       pil_image = Image.fromarray(np.uint8(npy_array))

#       # Get the filename without the extension
#       file_name = os.path.splitext(os.path.basename(npy_file))[0]

#       # Save the PIL image as a .jpg file in the output directory
#       pil_image.save(os.path.join(output_dir, file_name + '.jpg'))
#     except:
#       print(npy_file)
#       continue

/content/drive/MyDrive/DL_Project/examples/places/images_npy/image128.npy
/content/drive/MyDrive/DL_Project/examples/places/images_npy/image260.npy
/content/drive/MyDrive/DL_Project/examples/places/images_npy/image301.npy
/content/drive/MyDrive/DL_Project/examples/places/images_npy/image365.npy
/content/drive/MyDrive/DL_Project/examples/places/images_npy/image343.npy
/content/drive/MyDrive/DL_Project/examples/places/images_npy/image399.npy
/content/drive/MyDrive/DL_Project/examples/places/images_npy/image47.npy
/content/drive/MyDrive/DL_Project/examples/places/images_npy/image60.npy
/content/drive/MyDrive/DL_Project/examples/places/images_npy/image56.npy


In [ ]:
# ds = deeplake.load("hub://activeloop/places205")

# from torchvision import transforms

# tform = transforms.Compose([
#     transforms.ToTensor(), # Must convert to pytorch tensor for subsequent operations to run
# ])

# train_loader = ds.dataloader()\
#     .transform({'images': tform, 'labels': None})\
#     .batch(2)\
#     .shuffle()\
#     .pytorch(decode_method={'images': 'pil'})


# for i, sample in enumerate(train_loader):
#     if i==0:
#       print(i, sample)

In [ ]:
# import os
# import random
# from deeplake.core.tensor import Tensor
# import deeplake

# # Path to the directory containing the Places2 dataset
# dataset = deeplake.load("hub://activeloop/places205")

In [ ]:
# #Get the total number of images in the dataset

# random.seed(0)
# num_images = len(dataset)

# # Generate a random subset of indices
# subset_indices = random.sample(range(num_images), k=100)
# print(subset_indices)

# data_loader = torch.utils.data.DataLoader(dataset=dataset,
#                                                    batch_size=4,
#                                                    shuffle=False,
#                                                    num_workers=2,
#                                                    sampler = torch.utils.data.SubsetRandomSampler(subset_indices))


# data_loader
# # img = subset_data[0].images.numpy()
# # plt.imshow(img)
# # plt.show()

In [ ]:
# #Get the total number of images in the dataset

# def subset_data(self, subset_size = 100, seed = 0):
#   """Randomly subset data from the original Dataset"
#   subset_size : Length of subset
#   seed: Seed to reproduce the results
# """

#   random.seed(seed)
#   num_images = self.__len__()

#   # Generate a random subset of indices
#   subset_indices = random.sample(range(num_images), k=subset_size)

#   # Retrieve the images and labels corresponding to the subset indices
#   subset_data = []
#   for i in subset_indices:
#       img, label = self.data[i].images, self.data[i].labels
#       subset_data.append((img, label))

#   # Print the size of the subset
#   print("Size of the subset: ", len(subset_data))
  
#   return subset_data